## Libraries

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Dependencies and Setup
import time
from pprint import pprint
import requests
from datetime import date, timedelta, datetime
import json
from pprint import pprint
from tqdm import tqdm
from tqdm import tqdm_notebook
# sqlite Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

In [2]:
from nba_api.stats.endpoints import playercareerstats, drafthistory, commonplayerinfo, playerawards

In [3]:
# Not used
# cumestatsplayer,draftcombinedrillresults,playerdashboardbyteamperformance, leagueleaders, leaguedashplayerstats, draftcombinenonstationaryshooting, draftcombinestats, commonallplayers,

-------------

## Open Saved JSON

In [4]:
with open('./_players_all_data.json') as json_file:
    data = json.load(json_file)

In [5]:
#  Player positions
player_positions = pd.read_csv('../../datasets/nba_positions_no_team.csv') 

In [ ]:
#  Player positions
player_positions = pd.read_csv('../../datasets/nba_positions_no_team.csv') 

In [7]:
player_positions.avg()

AttributeError: 'DataFrame' object has no attribute 'avg'

--------------

#### Key: 

* GP: Games Played
* MIN: Minutes Played
* FGM: Field Goals Made
* FGA: Field Goals Attempted
* FG%: Field Goal Percentage
* 3PM: 3 Point Field Goals Made
* 3PA: 3 Point Field Goals Attempted
* 3P%: 3 Point Field Goals Percentage
* FTM: Free Throws Made
* FTA: Free Throws Attempted
* FT%: Free Throw Percentage
* OREB: Offensive Rebounds
* DREB: Defensive Rebounds
* REB: Rebounds
* AST: Assists
* TOV: Turnovers
* STL: Steals
* BLK: Blocks
* PF: Personal Fouls
* DD2: Double Doubles
* TD3: Trible Doubles
* PTS: Points
* YIL: Year in League




## Clean df and add columns

add Points Per Games Played (PPGP), add position

In [48]:
# df1.merge(df2, left_on='lkey', right_on='rkey',

#           suffixes=('_left', '_right'))

In [49]:
five_year_all_players = pd.read_json(data)
twenty_years_all_players_raw = five_year_all_players[five_year_all_players['SEASON_ID'] > '1999-00']
twenty_years_all_players_raw = twenty_years_all_players_raw.merge(player_positions, left_on='PLAYER_NAME', right_on='PLAYER_NAME',suffixes=('_left', '_right')).drop_duplicates( keep='first').reset_index(drop=True).dropna(how='any')
twenty_years_all_players_raw['PPGP'] = round(twenty_years_all_players_raw['PTS'] / twenty_years_all_players_raw['GP'], 2)
twenty_years_all_players_raw['POSITION'] = twenty_years_all_players_raw['POSITION'].str.strip()

In [52]:
# Create seperate dataframes for each position
twenty_years_all_players_raw['POSITION'].value_counts()

SG    985
C     985
SF    858
PF    856
PG    824
G     449
F     360
GF      5
NA      5
Name: POSITION, dtype: int64

## Save combined data to json

In [53]:
twenty_years_all_players_raw=twenty_years_all_players_raw.to_json(orient='records')

# SAVE: Player_position
with open(f'./clean_combined_data_for_ml.json', 'w') as fp:
    json.dump(twenty_years_all_players_raw, fp)

## Open combined data from json

In [57]:
with open('./clean_combined_data_for_ml.json') as json_file:
    data = json.load(json_file)

In [58]:
# Default DF. Do not touch.
default = pd.read_json(data).copy()

## Dataframes for all positions 

In [60]:
# SG
sg = default[default['POSITION'] == 'SG']

In [61]:
# C
c = default[default['POSITION'] == 'C']

In [62]:
# SF
sf = default[default['POSITION'] == 'SF']

In [63]:
# PF
pf = default[default['POSITION'] == 'PF']

In [64]:
# PG
pg = default[default['POSITION'] == 'PG']

In [65]:
# G
g = default[default['POSITION'] == 'G']

In [66]:
# F
f = default[default['POSITION'] == 'F']

------------